In [3]:
import pandas as pd
import numpy as np


In [4]:
import os

os.chdir(r"C:\Users\iyedm\OneDrive\Desktop\RecSys")


In [5]:
movies = pd.read_csv('./archive/movies.dat',delimiter="::", header=None, names=['MovieID','Title','Genres'], encoding='latin-1')
ratings = pd.read_csv('./archive/ratings.dat',delimiter="::", header=None, names=['UserID','MovieID','Rating','Timestamp'], encoding='latin-1')

C:\Users\iyedm\AppData\Local\Temp\ipykernel_7968\3566365147.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('./archive/movies.dat',delimiter="::", header=None, names=['MovieID','Title','Genres'], encoding='latin-1')
C:\Users\iyedm\AppData\Local\Temp\ipykernel_7968\3566365147.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('./archive/ratings.dat',delimiter="::", header=None, names=['UserID','MovieID','Rating','Timestamp'], encoding='latin-1')


## Content-Based Filtering

In [6]:
movies.head(5)

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings.head(5)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
ratings["UserID"].unique

<bound method Series.unique of 0             1
1             1
2             1
3             1
4             1
           ... 
1000204    6040
1000205    6040
1000206    6040
1000207    6040
1000208    6040
Name: UserID, Length: 1000209, dtype: int64>

In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  3883 non-null   int64 
 1   Title    3883 non-null   object
 2   Genres   3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [10]:
movies.shape

(3883, 3)

In [11]:
movies["Genres"] = movies["Genres"].str.replace("|", " ")

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
genre_matrix = tfidf.fit_transform(movies['Genres'])


In [13]:
movie_indexer = {k:v for k,v in enumerate(movies["MovieID"])}

In [14]:
movie_indexer

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 10,
 10: 11,
 11: 12,
 12: 13,
 13: 14,
 14: 15,
 15: 16,
 16: 17,
 17: 18,
 18: 19,
 19: 20,
 20: 21,
 21: 22,
 22: 23,
 23: 24,
 24: 25,
 25: 26,
 26: 27,
 27: 28,
 28: 29,
 29: 30,
 30: 31,
 31: 32,
 32: 33,
 33: 34,
 34: 35,
 35: 36,
 36: 37,
 37: 38,
 38: 39,
 39: 40,
 40: 41,
 41: 42,
 42: 43,
 43: 44,
 44: 45,
 45: 46,
 46: 47,
 47: 48,
 48: 49,
 49: 50,
 50: 51,
 51: 52,
 52: 53,
 53: 54,
 54: 55,
 55: 56,
 56: 57,
 57: 58,
 58: 59,
 59: 60,
 60: 61,
 61: 62,
 62: 63,
 63: 64,
 64: 65,
 65: 66,
 66: 67,
 67: 68,
 68: 69,
 69: 70,
 70: 71,
 71: 72,
 72: 73,
 73: 74,
 74: 75,
 75: 76,
 76: 77,
 77: 78,
 78: 79,
 79: 80,
 80: 81,
 81: 82,
 82: 83,
 83: 84,
 84: 85,
 85: 86,
 86: 87,
 87: 88,
 88: 89,
 89: 90,
 90: 92,
 91: 93,
 92: 94,
 93: 95,
 94: 96,
 95: 97,
 96: 98,
 97: 99,
 98: 100,
 99: 101,
 100: 102,
 101: 103,
 102: 104,
 103: 105,
 104: 106,
 105: 107,
 106: 108,
 107: 109,
 108: 110,
 109: 111,
 110: 1

In [15]:
def get_key_from_value(d, value):
    for k, v in d.items():
        if v == value:
            return k
    return None

In [16]:
genre_matrix.shape

(3883, 20)

In [17]:
def user_profile(user_id):
    user_movies = ratings[ratings['UserID'] == user_id]
    movies_ids = list(user_movies['MovieID'].astype(int).values)
    movies_idx = [get_key_from_value(movie_indexer, id) for id in movies_ids]
    ratings_values = user_movies['Rating'].values
    ratings_vecs = ratings_values[:, np.newaxis]
    movies_vecs = genre_matrix[movies_idx ,:]

    profile = movies_vecs.multiply(ratings_vecs).mean(axis=0)
    profile_array = np.array(profile)
    return profile_array

In [18]:
user_profile(1)

array([[0.20210395, 0.20259461, 0.95459791, 0.8904542 , 0.55052188,
        0.10597574, 0.        , 1.30334081, 0.15897464, 0.12636749,
        0.        , 0.        , 0.76645368, 0.        , 0.        ,
        0.25866158, 0.12636749, 0.14820802, 0.15886893, 0.        ]])

In [19]:
def cosine_similarity(a,b):
    return np.dot(a,b.T) / (np.linalg.norm(a) * np.linalg.norm(b))

In [20]:
def content_based(user_id, top_n=10):
    similarities = []
    for i in range(movies.shape[0]):
        user_prof = user_profile(user_id)
        movie_vector = genre_matrix[i,:].toarray()
        sim = cosine_similarity(user_prof, movie_vector)
        similarities.append((i, sim))
    sorted_sim= sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]
    recommended_movie_indices = [idx for idx, sim in sorted_sim]
    recommend_movies_ids = [movie_indexer[index] for index in recommended_movie_indices]
    return movies[movies["MovieID"].isin(recommend_movies_ids)]

In [21]:
user_id = 12
content_based(user_id, top_n=10)

,MovieID,Title,Genres
399,403,Two Crimes (1995),Comedy Crime Drama
1445,1473,Best Men (1997),Action Comedy Crime Drama
1745,1809,Hana-bi (1997),Comedy Crime Drama
1849,1918,Lethal Weapon 4 (1998),Action Comedy Crime Drama
1931,2000,Lethal Weapon (1987),Action Comedy Crime Drama
1932,2001,Lethal Weapon 2 (1989),Action Comedy Crime Drama
1933,2002,Lethal Weapon 3 (1992),Action Comedy Crime Drama
3115,3184,Montana (1998),Action Comedy Crime Drama
3197,3266,Man Bites Dog (C'est arrivé près de chez vous)...,Action Comedy Crime Drama
3293,3362,Dog Day Afternoon (1975),Comedy Crime Drama


## User-User

In [22]:
ratings.head(5)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [23]:
pivot_table = ratings.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)
pivot_table.head(5)

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df = pd.Series(ratings['UserID'].value_counts(), name= 'count')
df

UserID
4169    2314
1680    1850
4277    1743
1941    1595
1181    1521
        ... 
5725      20
3407      20
1664      20
4419      20
3021      20
Name: count, Length: 6040, dtype: int64

In [25]:
k_users = 50
k_movies = 50

top_k_users = ratings['UserID'].value_counts().nlargest(k_users).index
top_k_movies = ratings['MovieID'].value_counts().nlargest(k_movies).index

In [26]:
top_k_users

Index([4169, 1680, 4277, 1941, 1181,  889, 3618, 2063, 1150, 1015, 5795, 4344,
       1980, 2909, 1449, 4510,  424, 4227, 5831, 3841, 3391, 4508, 1088, 5367,
       3808,  549, 1285, 3224, 3539, 4543, 5643, 1448,  752, 3032, 3824,  524,
       4448, 4064, 4725, 1010, 5954,  678, 1447, 4447, 2116,  550, 1605, 3526,
       1698, 1880],
      dtype='int64', name='UserID')

In [27]:
ratings_topk = ratings[ratings['UserID'].isin(top_k_users) & ratings['MovieID'].isin(top_k_movies)]
ratings_topk.head(5)

,UserID,MovieID,Rating,Timestamp
62372,424,2987,4,976292802
62402,424,2997,4,978013182
62405,424,1259,4,997755265
62409,424,589,4,976281498
62411,424,1,4,976306643


In [28]:
ratings_topk.shape

(2317, 4)

In [29]:
ratings_topk.index

Index([ 62372,  62402,  62405,  62409,  62411,  62430,  62440,  62453,  62533,
        62542,
       ...
       985972, 985985, 986017, 986048, 986090, 986123, 986130, 986148, 986159,
       986179],
      dtype='int64', length=2317)

In [30]:
pivot_table = ratings_topk.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)
pivot_table.head(5)

MovieID,1,34,50,110,260,296,318,356,457,480,...,2628,2716,2762,2791,2858,2916,2987,2997,3175,3578
UserID,,,,,,,,,,,,,,,,,,,,,
424,4.0,4.0,5.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,...,4.0,5.0,4.0,5.0,5.0,4.0,4.0,4.0,5.0,4.0
524,5.0,4.0,0.0,5.0,5.0,5.0,5.0,4.0,5.0,0.0,...,4.0,5.0,3.0,5.0,5.0,4.0,5.0,5.0,0.0,4.0
549,5.0,4.0,5.0,3.0,5.0,5.0,5.0,2.0,5.0,4.0,...,5.0,5.0,4.0,5.0,0.0,5.0,4.0,4.0,0.0,3.0
550,5.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,5.0,...,3.0,5.0,0.0,3.0,4.0,5.0,5.0,5.0,5.0,4.0
678,5.0,5.0,5.0,4.0,5.0,2.0,5.0,4.0,5.0,4.0,...,4.0,3.0,5.0,4.0,5.0,4.0,5.0,4.0,4.0,4.0


In [31]:
pivot_table.shape

(50, 50)

In [32]:
user_indexer = {user_id: index for index, user_id in enumerate(pivot_table.index)}
movie_indexer = {movie_id: index for index, movie_id in enumerate(pivot_table.columns)}
user_indexer


{424: 0,
 524: 1,
 549: 2,
 550: 3,
 678: 4,
 752: 5,
 889: 6,
 1010: 7,
 1015: 8,
 1088: 9,
 1150: 10,
 1181: 11,
 1285: 12,
 1447: 13,
 1448: 14,
 1449: 15,
 1605: 16,
 1680: 17,
 1698: 18,
 1880: 19,
 1941: 20,
 1980: 21,
 2063: 22,
 2116: 23,
 2909: 24,
 3032: 25,
 3224: 26,
 3391: 27,
 3526: 28,
 3539: 29,
 3618: 30,
 3808: 31,
 3824: 32,
 3841: 33,
 4064: 34,
 4169: 35,
 4227: 36,
 4277: 37,
 4344: 38,
 4447: 39,
 4448: 40,
 4508: 41,
 4510: 42,
 4543: 43,
 4725: 44,
 5367: 45,
 5643: 46,
 5795: 47,
 5831: 48,
 5954: 49}

In [33]:
from scipy.sparse import csr_matrix
pivot_filled = pivot_table.fillna(0)
X = csr_matrix(pivot_filled.values)   # shape (n_users, n_items)
X

<50x50 sparse matrix of type '<class 'numpy.float64'>'
	with 2317 stored elements in Compressed Sparse Row format>

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

user_sim = cosine_similarity(X)
user_sim

array([[1.        , 0.92199952, 0.95026606, ..., 0.95985362, 0.97109979,
        0.88482692],
       [0.92199952, 1.        , 0.89028751, ..., 0.90992034, 0.93337566,
        0.86163812],
       [0.95026606, 0.89028751, 1.        , ..., 0.94050465, 0.93425558,
        0.88759274],
       ...,
       [0.95985362, 0.90992034, 0.94050465, ..., 1.        , 0.94333524,
        0.8840384 ],
       [0.97109979, 0.93337566, 0.93425558, ..., 0.94333524, 1.        ,
        0.85918341],
       [0.88482692, 0.86163812, 0.88759274, ..., 0.8840384 , 0.85918341,
        1.        ]])

In [35]:
def get_key_from_value(d, value):
    for k, v in d.items():
        if v == value:
            return k
    return None


In [36]:
user_id  = np.random.choice(top_k_users)
print(user_id)

top_n = 10
#most similar users
similarity = np.sort(-user_sim[user_indexer[user_id]])[1: top_n +1]
print(similarity)
similar_users = [get_key_from_value(user_indexer, i ) for i in np.argsort(-user_sim[user_indexer[user_id]])[1: top_n +1]]
similar_users

4447
[-0.90326738 -0.90226947 -0.90217057 -0.89830401 -0.8964009  -0.89441149
 -0.8882631  -0.88727989 -0.8856712  -0.88566146]


[3391, 4448, 549, 4508, 424, 1449, 4277, 4543, 4169, 4344]

In [37]:
movies_watched_by_similar_users = ratings_topk[ratings_topk['UserID'].isin(similar_users)]
movies_watched_by_similar_users.head(10)


,UserID,MovieID,Rating,Timestamp
62372,424,2987,4,976292802
62402,424,2997,4,978013182
62405,424,1259,4,997755265
62409,424,589,4,976281498
62411,424,1,4,976306643
62430,424,1265,4,993226351
62440,424,593,5,976307032
62453,424,1270,5,978012869
62533,424,919,5,976292745
62542,424,924,5,976281418


In [38]:
movies_of_similar_users_non_watched_by_user = movies_watched_by_similar_users[~movies_watched_by_similar_users['MovieID'].isin(ratings_topk[ratings_topk['UserID'] == user_id]['MovieID'])]
movies_of_similar_users_non_watched_by_user.head(10)

,UserID,MovieID,Rating,Timestamp
62411,424,1,4,976306643
63076,424,2355,5,978008948
63104,424,3175,5,978013345
63126,424,110,4,976292974
63149,424,2396,4,976292548
63336,424,2762,4,976308387
63374,424,3578,4,976398200
63446,424,527,4,976292891
63535,424,2028,4,976292950
82596,549,1,5,976070010


In [39]:
non_watched_movies_nb = movies_of_similar_users_non_watched_by_user['MovieID'].nunique()
non_watched_movies_nb

9

In [40]:
recommended_movies = movies_of_similar_users_non_watched_by_user.groupby('MovieID')["Rating"].mean().sort_values(ascending=False).head(10) 
recommended_movies = recommended_movies.to_frame().reset_index()
recommended_movies.merge(movies, on='MovieID')

,MovieID,Rating,Title,Genres
0,527,4.750000,Schindler's List (1993),Drama War
1,1,4.500000,Toy Story (1995),Animation Children's Comedy
2,2355,4.500000,"Bug's Life, A (1998)",Animation Children's Comedy
3,2762,4.500000,"Sixth Sense, The (1999)",Thriller
4,3175,4.166667,Galaxy Quest (1999),Adventure Comedy Sci-Fi
5,110,4.100000,Braveheart (1995),Action Drama War
6,2028,4.000000,Saving Private Ryan (1998),Action Drama War
7,2396,4.000000,Shakespeare in Love (1998),Comedy Romance
8,3578,3.900000,Gladiator (2000),Action Drama


In [41]:
def user_user_based_recommendation(user_id, top_n=10, k_users=50, k_movies=50):
    k_users = 50
    k_movies = 50
    top_k_users = ratings['UserID'].value_counts().nlargest(k_users).index
    top_k_movies = ratings['MovieID'].value_counts().nlargest(k_movies).index
    ratings_topk = ratings[ratings['UserID'].isin(top_k_users) & ratings['MovieID'].isin(top_k_movies)]
    pivot_table = ratings_topk.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)
    user_indexer = {user_id: index for index, user_id in enumerate(pivot_table.index)}
    pivot_filled = pivot_table.fillna(0)
    X = csr_matrix(pivot_filled.values)   # shape (n_users, n_items)
    user_sim = cosine_similarity(X)

    def get_key_from_value(d, value):
        for k, v in d.items():
            if v == value:
                return k
        return None
    
    #most similar users
    similar_users = [get_key_from_value(user_indexer, i ) for i in np.argsort(-user_sim[user_indexer[user_id]])[1: top_n +1]]
    movies_watched_by_similar_users = ratings_topk[ratings_topk['UserID'].isin(similar_users)]
    movies_of_similar_users_non_watched_by_user = movies_watched_by_similar_users[~movies_watched_by_similar_users['MovieID'].isin(ratings_topk[ratings_topk['UserID'] == user_id]['MovieID'])]
    recommended_movies = movies_of_similar_users_non_watched_by_user.groupby('MovieID')["Rating"].mean().sort_values(ascending=False).head(10) 
    recommended_movies = recommended_movies.to_frame().reset_index()
    rec_table = recommended_movies.merge(movies, on='MovieID')
    return rec_table






In [42]:
random_user=  np.random.choice(top_k_users)
user_user_based_recommendation(random_user)

,MovieID,Rating,Title,Genres
0,527,4.600000,Schindler's List (1993),Drama War
1,2396,4.111111,Shakespeare in Love (1998),Comedy Romance


## Item-Item 

In [43]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [44]:
k_users = 50
k_movies = 3000
top_k_users = ratings['UserID'].value_counts().nlargest(k_users).index
top_k_movies = ratings['MovieID'].value_counts().nlargest(k_movies).index
ratings_topk = ratings[ratings['UserID'].isin(top_k_users) & ratings['MovieID'].isin(top_k_movies)]
pivot_table = ratings_topk.pivot(index='MovieID', columns='UserID', values='Rating').fillna(0)

In [45]:
pivot_table.head(10)

UserID,424,524,549,550,678,752,889,1010,1015,1088,...,4448,4508,4510,4543,4725,5367,5643,5795,5831,5954
MovieID,,,,,,,,,,,,,,,,,,,,,
1,4.0,5.0,5.0,5.0,5.0,5.0,4.0,2.0,5.0,4.0,...,4.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,4.0,0.0,3.0,4.0,4.0,0.0,0.0,1.0,3.0,0.0,...,3.0,3.0,2.0,0.0,3.0,2.0,3.0,2.0,3.0,4.0
3,0.0,2.0,1.0,3.0,0.0,2.0,2.0,1.0,0.0,3.0,...,0.0,1.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,4.0,4.0,0.0
5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0
6,4.0,5.0,5.0,4.0,0.0,2.0,0.0,1.0,0.0,4.0,...,5.0,3.0,4.0,5.0,0.0,4.0,4.0,4.0,4.0,0.0
7,0.0,4.0,3.0,3.0,0.0,0.0,2.0,0.0,2.0,3.0,...,0.0,0.0,0.0,3.0,3.0,0.0,3.0,3.0,4.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
9,0.0,2.0,3.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0


In [46]:
pivot_filled = pivot_table.fillna(0)
X = csr_matrix(pivot_filled.values)   # shape (n_users, n_items)
item_sim = cosine_similarity(X)
item_sim

array([[1.        , 0.81042563, 0.63924353, ..., 0.38788532, 0.13392991,
        0.59426683],
       [0.81042563, 1.        , 0.59079477, ..., 0.39107694, 0.        ,
        0.52166317],
       [0.63924353, 0.59079477, 1.        , ..., 0.21004201, 0.14142136,
        0.39986283],
       ...,
       [0.38788532, 0.39107694, 0.21004201, ..., 1.        , 0.        ,
        0.54328867],
       [0.13392991, 0.        , 0.14142136, ..., 0.        , 1.        ,
        0.1119785 ],
       [0.59426683, 0.52166317, 0.39986283, ..., 0.54328867, 0.1119785 ,
        1.        ]])

In [47]:
random_user=  np.random.choice(top_k_users)
watched_movies_tuples = [
    (row.MovieID, row.Rating)
    for row in ratings_topk[ratings_topk['UserID'] == random_user].itertuples()
]

watched_movies_tuples_sorted = sorted(watched_movies_tuples, key=lambda x: x[0])

print(len(watched_movies_tuples_sorted))


1794


In [48]:
item_indexer = {movie_id: index for index, movie_id in enumerate(pivot_table.index)}
item_indexer

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 34: 32,
 35: 33,
 36: 34,
 38: 35,
 39: 36,
 40: 37,
 41: 38,
 42: 39,
 43: 40,
 44: 41,
 45: 42,
 46: 43,
 47: 44,
 48: 45,
 49: 46,
 50: 47,
 52: 48,
 54: 49,
 55: 50,
 57: 51,
 58: 52,
 60: 53,
 61: 54,
 62: 55,
 63: 56,
 64: 57,
 65: 58,
 66: 59,
 68: 60,
 69: 61,
 70: 62,
 71: 63,
 72: 64,
 73: 65,
 74: 66,
 76: 67,
 77: 68,
 78: 69,
 79: 70,
 80: 71,
 81: 72,
 82: 73,
 83: 74,
 85: 75,
 86: 76,
 87: 77,
 88: 78,
 89: 79,
 92: 80,
 93: 81,
 94: 82,
 95: 83,
 97: 84,
 99: 85,
 100: 86,
 101: 87,
 102: 88,
 103: 89,
 104: 90,
 105: 91,
 107: 92,
 110: 93,
 111: 94,
 112: 95,
 113: 96,
 116: 97,
 117: 98,
 118: 99,
 121: 100,
 122: 101,
 123: 102,
 125: 103,
 126: 104,
 132: 105,
 135: 106,
 140: 107,
 141: 108,
 144: 

In [49]:
movies_per_movie = 2
similar_movies = set()
for i in range(len(watched_movies_tuples)):
    similar_movies.update([get_key_from_value(item_indexer, i ) for i in np.argsort(-item_sim[item_indexer[watched_movies_tuples[i][0]]])[1: movies_per_movie +1]])

watched_movies = [movie for (movie, _) in watched_movies_tuples]
unwatched_similar_movies = [movie for movie in similar_movies if movie not in watched_movies]
len(unwatched_similar_movies)

220

In [50]:
def predict_movie_rating(movie_id):
    rating = 0
    sim_sum = 0
    for i in range(len(watched_movies_tuples)):
        similarity = item_sim[item_indexer[movie_id], item_indexer[watched_movies_tuples[i][0]]]
        sim_sum += similarity
        rating += similarity * watched_movies_tuples[i][1]
    return rating / sim_sum
    

In [51]:
predicted_ratings = [
    (mid, predict_movie_rating(mid))
    for mid in similar_movies
]

predicted_ratings = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)

print(predicted_ratings)


[(2689, 3.8134995154570257), (1583, 3.813254918226963), (2853, 3.813254918226963), (341, 3.810269019336089), (1865, 3.804920133124045), (3680, 3.7949744620037986), (2483, 3.7916591210035984), (363, 3.789862327673232), (3679, 3.788573073958609), (1236, 3.7778289879064237), (278, 3.776970764045993), (1151, 3.7764550517185507), (77, 3.7751662966312667), (3218, 3.7743169227224467), (3939, 3.772771394412996), (1322, 3.7708050299170783), (30, 3.7698357289072897), (1312, 3.7698357289072897), (1572, 3.7649914564202236), (206, 3.7641714197870155), (3938, 3.76339410716149), (3077, 3.7631515894), (219, 3.762487594425986), (639, 3.762472187247377), (302, 3.761976878176913), (669, 3.7616898774705607), (846, 3.7600232463595455), (121, 3.759387387178798), (2465, 3.7587811680399774), (3941, 3.758591158478315), (2351, 3.7581487253912482), (1175, 3.7581082752865975), (3665, 3.7577923550328465), (501, 3.7565850836101795), (392, 3.7557358008827832), (841, 3.755150159531281), (3833, 3.7547169548907515), (3

In [52]:
predictions_df = pd.DataFrame(predicted_ratings, columns=['MovieID', 'PredictedRating'])
predictions_df


,MovieID,PredictedRating
0,2689,3.813500
1,1583,3.813255
2,2853,3.813255
3,341,3.810269
4,1865,3.804920
...,...,...
1467,470,3.333877
1468,656,3.329702
1469,1825,3.325228
1470,3945,3.313077


In [53]:
predictions_df = predictions_df.merge(movies, on='MovieID')
predictions_df

,MovieID,PredictedRating,Title,Genres
0,2689,3.813500,Get Bruce (1999),Documentary
1,1583,3.813255,"Simple Wish, A (1997)",Children's Fantasy
2,2853,3.813255,"Communion (a.k.a. Alice, Sweet Alice/Holy Terr...",Horror
3,341,3.810269,Double Happiness (1994),Drama
4,1865,3.804920,Wild Man Blues (1998),Documentary
...,...,...,...,...
1467,470,3.333877,House Party 3 (1994),Comedy
1468,656,3.329702,Eddie (1996),Comedy
1469,1825,3.325228,"Player's Club, The (1998)",Action Drama
1470,3945,3.313077,Digimon: The Movie (2000),Adventure Animation Children's


In [54]:
def item_item_based_recommendation(user_id, movies_per_movie=2, k_users=50, k_movies=movies.shape[0]):
    top_k_users = ratings['UserID'].value_counts().nlargest(k_users).index
    top_k_movies = ratings['MovieID'].value_counts().nlargest(k_movies).index
    ratings_topk = ratings[ratings['UserID'].isin(top_k_users) & ratings['MovieID'].isin(top_k_movies)]
    pivot_table = ratings_topk.pivot(index='MovieID', columns='UserID', values='Rating').fillna(0)
    pivot_filled = pivot_table.fillna(0)
    X = csr_matrix(pivot_filled.values)   # shape (n_users, n_items)
    item_sim = cosine_similarity(X)
    item_indexer = {movie_id: index for index, movie_id in enumerate(pivot_table.index)}
    
    watched_movies_tuples = [
        (row.MovieID, row.Rating)
        for row in ratings[ratings['UserID'] == user_id].itertuples()
    ]
    
    similar_movies = set()
    for i in range(len(watched_movies_tuples)):
        similar_movies.update([get_key_from_value(item_indexer, i ) for i in np.argsort(-item_sim[item_indexer[watched_movies_tuples[i][0]]])[1: movies_per_movie +1]])
    
    watched_movies = [movie for (movie, _) in watched_movies_tuples]
    unwatched_similar_movies = [movie for movie in similar_movies if movie not in watched_movies]
    
    def predict_movie_rating(movie_id):
        rating = 0
        sim_sum = 0
        for i in range(len(watched_movies_tuples)):
            similarity = item_sim[item_indexer[movie_id], item_indexer[watched_movies_tuples[i][0]]]
            sim_sum += similarity
            rating += similarity * watched_movies_tuples[i][1]
        return rating / sim_sum
    
    predicted_ratings = [
        (mid, predict_movie_rating(mid))
        for mid in unwatched_similar_movies
    ]
    
    predicted_ratings = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)
    predictions_df = pd.DataFrame(predicted_ratings, columns=['MovieID', 'PredictedRating'])
    predictions_df = predictions_df.merge(movies, on='MovieID')
    return predictions_df


In [55]:
random_user=  np.random.choice(top_k_users)
item_item_based_recommendation(random_user)

,MovieID,PredictedRating,Title,Genres
0,568,4.092599,Bhaji on the Beach (1993),Comedy Drama
1,3854,4.092599,Aimée & Jaguar (1999),Drama Romance
2,2843,4.082724,"Black Cat, White Cat (Crna macka, beli macor) ...",Comedy Romance
3,2839,4.064108,West Beirut (West Beyrouth) (1998),Drama
4,3872,4.039868,"Suddenly, Last Summer (1959)",Drama
...,...,...,...,...
211,461,3.861255,Go Fish (1994),Drama Romance
212,3777,3.856360,Nekromantik (1987),Comedy Horror
213,840,3.843182,House Arrest (1996),Comedy
214,470,3.804886,House Party 3 (1994),Comedy


In [56]:
item_item_based_recommendation(14)

,MovieID,PredictedRating,Title,Genres
0,3739,3.604478,Trouble in Paradise (1932),Comedy Romance
1,2238,3.575348,Seven Beauties (Pasqualino Settebellezze) (1976),Comedy Drama
2,2425,3.491302,"General, The (1998)",Crime
3,3813,3.451506,Interiors (1978),Drama
4,1183,3.394787,"English Patient, The (1996)",Drama Romance War
5,3504,3.392424,Network (1976),Comedy Drama
6,2324,3.374651,Life Is Beautiful (La Vita è bella) (1997),Comedy Drama
7,247,3.371243,Heavenly Creatures (1994),Drama Fantasy Romance Thriller
8,2289,3.364644,"Player, The (1992)",Comedy Drama
9,2248,3.361308,Say Anything... (1989),Comedy Drama Romance


## FunkSVD

In [57]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [58]:
num_users_to_take = 100
num_movies_to_take = 30

In [59]:
ratings_grouped = ratings.groupby("UserID")
def filter_ratings(df: pd.DataFrame):
    return df.nlargest(num_movies_to_take, "Timestamp")
ratings_filter_movies = ratings_grouped.apply(filter_ratings)
ratings_filter_users = ratings_filter_movies[ratings_filter_movies["UserID"]<=num_users_to_take].reset_index(drop= True)

C:\Users\iyedm\AppData\Local\Temp\ipykernel_7968\686613092.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ratings_filter_movies = ratings_grouped.apply(filter_ratings)


In [60]:
ratings_filter_users.shape

(2905, 4)

In [61]:
ratings_filter_users.head()

,UserID,MovieID,Rating,Timestamp
0,1,48,5,978824351
1,1,1566,4,978824330
2,1,1907,4,978824330
3,1,2355,5,978824291
4,1,2294,4,978824291


In [62]:
ratings_grouped = ratings_filter_users.groupby("UserID")
def leave_one_out(df:pd.DataFrame):
    return df.nlargest(1, "Timestamp")
leave_one_out_df = ratings_grouped.apply(leave_one_out)
leave_one_out_df

C:\Users\iyedm\AppData\Local\Temp\ipykernel_7968\1378189954.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  leave_one_out_df = ratings_grouped.apply(leave_one_out)


,,UserID,MovieID,Rating,Timestamp
UserID,,,,,
1,0,1,48,5,978824351
2,30,2,1687,3,978300174
3,60,3,2081,4,978298504
4,90,4,1954,5,978294282
5,111,5,288,2,978246585
...,...,...,...,...,...
96,2765,96,1073,4,980565244
97,2795,97,3334,4,999717170
98,2825,98,318,5,977619823


In [63]:
ratings_filter_users.shape

(2905, 4)

In [64]:
loo_indices = leave_one_out_df.index.get_level_values(1)
loo_indices


Index([   0,   30,   60,   90,  111,  141,  171,  201,  231,  261,  291,  321,
        344,  374,  399,  429,  459,  489,  519,  549,  573,  595,  625,  655,
        685,  715,  745,  775,  805,  835,  865,  895,  925,  955,  985, 1015,
       1045, 1075, 1105, 1135, 1165, 1190, 1220, 1244, 1274, 1304, 1334, 1356,
       1386, 1416, 1446, 1476, 1506, 1536, 1566, 1591, 1621, 1651, 1681, 1711,
       1741, 1771, 1801, 1831, 1858, 1888, 1914, 1944, 1974, 2004, 2034, 2063,
       2093, 2123, 2153, 2183, 2213, 2243, 2273, 2303, 2333, 2363, 2393, 2423,
       2453, 2483, 2513, 2543, 2573, 2594, 2624, 2654, 2684, 2714, 2735, 2765,
       2795, 2825, 2845, 2875],
      dtype='int64')

In [65]:
leave_one_out_df = leave_one_out_df.reset_index(drop= True)

train_df = ratings_filter_users.drop(loo_indices)

train_df.head()

,UserID,MovieID,Rating,Timestamp
1,1,1566,4,978824330
2,1,1907,4,978824330
3,1,2355,5,978824291
4,1,2294,4,978824291
5,1,783,4,978824291


In [66]:
train_df.shape

(2805, 4)

In [67]:
leave_one_out_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,48,5,978824351
1,2,1687,3,978300174
2,3,2081,4,978298504
3,4,1954,5,978294282
4,5,288,2,978246585


In [68]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)  # outputs 'cuda' if GPU available, else 'cpu'


cuda


In [69]:
num_users = ratings_filter_users['UserID'].nunique()
num_movies = ratings_filter_users['MovieID'].nunique()
num_users, num_movies

(100, 1311)

In [70]:
R_train = train_df.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)
R_train = torch.tensor(R_train.values, dtype=torch.float32).to(device)
R_train.shape


torch.Size([100, 1284])

In [71]:
R_train

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 3.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [72]:
num_users_train, num_movies_train = R_train.shape 
latent_dim = 20

U = 0.01 * torch.rand((num_users_train, latent_dim), device=device, dtype=torch.float32)
V = 0.01 * torch.rand((num_movies_train, latent_dim), device=device, dtype=torch.float32)



In [73]:
R_train.shape

torch.Size([100, 1284])

In [74]:
import torch

R_train = R_train.clone()  # avoid modifying the original
Mask = R_train != 0       # mask of known ratings

# Compute per-user mean and std over known ratings only
sum_ratings = (R_train * Mask).sum(dim=1)
count_ratings = Mask.sum(dim=1)
mean = sum_ratings / count_ratings.clamp(min=1)  # avoid division by 0

# Compute std over known ratings
diff = (R_train - mean.unsqueeze(1)) * Mask
std = torch.sqrt((diff**2).sum(dim=1) / count_ratings.clamp(min=1))
std[std == 0] = 1.0  # avoid division by zero

# Normalize only known ratings
R_norm = torch.zeros_like(R_train)
R_norm[Mask] = ((R_train - mean.unsqueeze(1)) / std.unsqueeze(1))[Mask]

R_train = R_norm  # replace original with normalized


In [ ]:
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
epochs = 1000
learning_rate = 0.01
lambda_reg = 0.02
Loss = 0

# Move tensors to GPU
R_train = R_train.to(device)
U = U.to(device)
V = V.to(device)
Mask = (R_train != 0).float().to(device)

for epoch in range(epochs):
    R_hat = U @ V.T
    Error = (R_train - R_hat) * Mask
    Loss = torch.sum(Error**2)
    print(f"Epoch {epoch} : Loss {Loss.item():.4f}")

    for i in range(Error.shape[0]):
        for j in range(Error.shape[1]):
            if Mask[i,j] == 0:
                continue
            e_ij = Error[i,j]
            U_i_old = U[i, :].clone()      # stays on GPU
            U[i, :] += learning_rate * (e_ij * V[j, :] - lambda_reg * U[i, :])
            V[j, :] += learning_rate * (e_ij * U_i_old - lambda_reg * V[j, :])


## ALS

In [ ]:
def ALS_update(U, V, R, Mask, lambda_reg, axis='user'):
    """
    Vectorized ALS update per user or per item
    axis='user': update U given V
    axis='item': update V given U
    """
    k = U.shape[1] if axis=='user' else V.shape[1]
    I = torch.eye(k, device=device) * lambda_reg

    if axis == 'user':
        # U update
        for u in range(R.shape[0]):
            idx_items = Mask[u, :] > 0
            V_i = V[idx_items, :]
            R_u = R[u, idx_items].unsqueeze(1)
            A = V_i.T @ V_i + I
            b = V_i.T @ R_u
            U[u, :] = torch.linalg.solve(A, b).squeeze()
    else:
        # V update
        for i in range(R.shape[1]):
            idx_users = Mask[:, i] > 0
            U_u = U[idx_users, :]
            R_i = R[idx_users, i].unsqueeze(1)
            A = U_u.T @ U_u + I
            b = U_u.T @ R_i
            V[i, :] = torch.linalg.solve(A, b).squeeze()
    return U, V


In [ ]:
epochs= 10
lambda_reg=  0.02
for epoch in range(epochs):
    U, V = ALS_update(U, V, R_train, Mask, lambda_reg, axis='user')
    U, V = ALS_update(U, V, R_train, Mask, lambda_reg, axis='item')

    # Compute loss
    R_hat = U @ V.T
    Error = (R_train - R_hat) * Mask
    Loss = torch.sum(Error**2).item()
    print(f"Epoch {epoch}: Loss = {Loss:.4f}")


Epoch 0: Loss = 0.0279
Epoch 1: Loss = 0.0260
Epoch 2: Loss = 0.0247
Epoch 3: Loss = 0.0238
Epoch 4: Loss = 0.0231
Epoch 5: Loss = 0.0226
Epoch 6: Loss = 0.0222
Epoch 7: Loss = 0.0220
Epoch 8: Loss = 0.0217
Epoch 9: Loss = 0.0215


In [ ]:
leave_one_out_df

,UserID,MovieID,Rating,Timestamp
0,1,48,5,978824351
1,2,1687,3,978300174
2,3,2081,4,978298504
3,4,1954,5,978294282
4,5,288,2,978246585
...,...,...,...,...
95,96,1073,4,980565244
96,97,3334,4,999717170
97,98,318,5,977619823
98,99,2144,5,994354345


In [ ]:
test_df = leave_one_out_df[leave_one_out_df["MovieID"].isin(train_df["MovieID"].unique())]
test_df

,UserID,MovieID,Rating,Timestamp
0,1,48,5,978824351
2,3,2081,4,978298504
3,4,1954,5,978294282
4,5,288,2,978246585
5,6,597,5,978239019
...,...,...,...,...
94,95,3147,4,977629537
95,96,1073,4,980565244
96,97,3334,4,999717170
97,98,318,5,977619823


In [ ]:
user_indexer = {key: value for key, value in enumerate(train_df["UserID"])}
movie_indexer = {key: value for key, value in enumerate(train_df["MovieID"])}

In [ ]:
print("U device:", U.device, "V device:", V.device)


U device: cuda:0 V device: cuda:0


In [ ]:
user_id = 1     
movie_id = 48

user_index=  get_key_from_value(user_indexer, user_id)
movie_index = get_key_from_value(movie_indexer, movie_id)

print(user_index)
print(movie_index)
# Assuming U and V are currently on the GPU (CUDA device)
U_float = U.float()
V_float = V.float()
print(U.dtype)
print(V.dtype)
# 10 pred_norm = U[user_index, :] @ V[movie_index, :].T
pred_norm = U_float[user_index, :] @ V_float[movie_index, :].T
# Denormalize
pred_real = pred_norm * std[user_index] + mean[user_index]

print(f"Predicted rating for user {user_id}, movie {movie_id}: {pred_real.item():.2f}")


0
164
torch.float32
torch.float32
Predicted rating for user 1, movie 48: 4.83


C:\Users\iyedm\AppData\Local\Temp\ipykernel_20228\1465333683.py:15: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4419.)
  pred_norm = U_float[user_index, :] @ V_float[movie_index, :].T


## NMF

In [ ]:
a= torch.Tensor([[1,2],
                 [3,4]])

b= a * a
b

tensor([[ 1.,  4.],
        [ 9., 16.]])

In [ ]:
R_train.shape

torch.Size([100, 1284])

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

n_users, n_items = R_train.shape
k = 20
epochs = 100000
epsilon = 1e-5

# Initialize positive matrices
U = torch.rand((n_users, k), device=device, dtype=torch.float32)
V = torch.rand((n_items, k), device=device, dtype=torch.float32)

# Mask for known ratings
Mask = (R_train != 0).float().to(device)

for epoch in range(epochs):
    # Update V
    numerator_V = U.T @ (R_train * Mask)             # shape: (k, n_items)
    denominator_V = (U.T @ U) @ V.T + epsilon       # shape: (k, n_items)
    V = V * (numerator_V / denominator_V).T         # transpose to get back (n_items x k)

    # Update U
    numerator_U = (R_train * Mask) @ V              # shape: (n_users, k)
    denominator_U = U @ (V.T @ V) + epsilon        # shape: (n_users, k)
    U = U * (numerator_U / denominator_U)

    # Compute loss only on known entries
    R_hat = U @ V.T
    Error = (R_train - R_hat) * Mask
    loss = torch.sum(Error**2)

    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}: Loss={loss.item():.4f}")


Epoch 000: Loss=2642.9268
Epoch 010: Loss=2442.2471
Epoch 020: Loss=2647.4294
Epoch 030: Loss=2414.5786
Epoch 040: Loss=10473.2148
Epoch 050: Loss=2298.2002
Epoch 060: Loss=8855.4561
Epoch 070: Loss=2383.3228
Epoch 080: Loss=98316.1094
Epoch 090: Loss=2394.1160
Epoch 100: Loss=2302.1045
Epoch 110: Loss=2358.5361
Epoch 120: Loss=2371.3848
Epoch 130: Loss=2332.3188
Epoch 140: Loss=2452.8740
Epoch 150: Loss=2270.0991
Epoch 160: Loss=2229.5798
Epoch 170: Loss=2199.3784
Epoch 180: Loss=2231.6714
Epoch 190: Loss=2212.3008
Epoch 200: Loss=2246.2920
Epoch 210: Loss=2213.2793
Epoch 220: Loss=2269.7722
Epoch 230: Loss=2174.1782
Epoch 240: Loss=2166.9258
Epoch 250: Loss=2189.8801
Epoch 260: Loss=2321.2656
Epoch 270: Loss=2362.0786
Epoch 280: Loss=2306.6555
Epoch 290: Loss=2328.8833
Epoch 300: Loss=2291.0835
Epoch 310: Loss=2281.4065
Epoch 320: Loss=2270.6777
Epoch 330: Loss=2265.6890
Epoch 340: Loss=2495.2664
Epoch 350: Loss=2222.2061
Epoch 360: Loss=2207.5884
Epoch 370: Loss=2161.0693
Epoch 380: